In [ ]:





Logistics Supply Chain Delay Prediction¶By Jack Motta¶Table of Contents¶
Introduction
Background
Methods
Results
Discussion
Conclusion
References
Data Sources





In [ ]:





Introduction¶Project Overview¶For this machine learning project, I have chosen to focus on the logistics and supply chain industry, specifically predicting delivery outcomes—whether an order will arrive early, on time, or be delayed. I am particularly interested in this area because supply chain efficiency is a critical component of operational success for businesses, and delays in logistics can have significant financial and customer satisfaction consequences. With the increased reliance on e-commerce and global distribution networks, developing models that can proactively forecast potential delays can provide tremendous value to companies and customers alike.
The primary dataset for this project is a comprehensive logistics and supply chain dataset obtained from Kaggle. It contains 15,549 records and 41 features, capturing detailed information about customer orders, products, geographic regions, shipping modes, and delivery outcomes. The data spans from 2014 to 2018, providing a rich historical context for analyzing shipping performance across different time periods and regions. In addition to this, I have identified a secondary dataset from the U.S. Energy Information Administration that includes weekly U.S. diesel prices from 1993 through 2025. By integrating fuel cost data with the supply chain records, I hope to explore whether fluctuations in diesel prices correlate with shipping delays or timing.
The supply chain dataset includes both numerical and categorical variables that cover various aspects of each order. These include sales and profit metrics, customer and order location data, product and shipping information, and a target variable labeled as label, which indicates whether an order was delivered early (-1), on time (0), or delayed (1). This target variable serves as the foundation for the supervised learning model I will build.
The problem I aim to solve with this dataset is a multi-class classification task: predicting delivery outcomes based on pre-shipping information. My goal is to uncover which factors—such as shipping mode, region, order characteristics, and possibly external data like diesel prices—have the strongest influence on whether an order arrives late, early, or on time. Accurately predicting these outcomes can help businesses make informed decisions to improve logistical planning, allocate resources more efficiently, and ultimately enhance customer satisfaction.





In [ ]:





Background¶Importance of the Problem & Background Knowledge¶Predicting delivery outcomes—whether an order arrives early, on time, or late—is a critical issue in modern supply chain management. Timely deliveries are essential not only for customer satisfaction but also for reducing operational costs, maintaining efficient inventory systems, and ensuring smooth business workflows. As global supply chains become more complex, the ability to anticipate delivery timing using predictive models has gained significant strategic value.
Delivery performance is influenced by various factors, with lead time variability being particularly impactful. Inconsistent lead times increase uncertainty and can disrupt downstream logistics operations. Organizations with better control over lead time variability tend to perform more efficiently and manage risk more effectively (Rokoss et al., 2024). Transportation-related disruptions also significantly influence delivery outcomes. External factors such as traffic congestion, extreme weather events, and fluctuating fuel prices can cause delays even in otherwise stable supply chains (Ivanov & Dolgui, 2020). These disruptions not only increase delivery time but also reduce the reliability of shipping estimates, which can impact service-level agreements and customer expectations. Another major theme in this area is supply chain resilience—the capacity of a logistics system to adapt to unforeseen disruptions and recover quickly. Predictive analytics, including machine learning, play an essential role in building this resilience by helping organizations anticipate problems and act proactively (Gabellini et al., 2024). Machine learning models are particularly powerful tools for this kind of predictive work. When trained on large, diverse datasets that include historical delivery information and external variables such as diesel prices, these models can identify patterns and generate more accurate forecasts. Recent research shows that machine learning methods can improve decision-making across the supply chain, from demand forecasting to final-mile delivery prediction (Pazoki et al., 2023).
Overall, the ability to predict delivery timing is a high-impact challenge at the intersection of logistics, transportation, and data science. By applying machine learning to real-world supply chain data, this project contributes to a vital area of research and practice.




In [ ]:





Table 1: Data Dictionary



Original Column (Relabeled Column)
Type
Description




payment_type
Categorical
Method of payment used for the order.


profit_per_order (pre_discount_profit_per_order)
Numerical
Net income generated from each order before discount.


sales_per_customer (total_sales_per_customer)
Numerical
Total revenue attributed to a single customer.


category_id
Numerical
Numeric identifier for the product category.


category_name
Text
Name of the category to which the product belongs.


customer_city (customer_purchase_city)
Categorical
City where the customer made the purchase.


customer_country (customer_purchase_country)
Categorical
Country where the customer made the purchase.


customer_id
Numerical
Unique identifier for the customer.


customer_segment
Categorical
Classification of customer type (e.g., Consumer, Corporate).


customer_state (store_location_state)
Categorical
State in which the store processing the order is located.


customer_zipcode
Text
Zip code of the customer's location.


department_id
Numerical
Numeric ID for the store department.


department_name
Text
Name of the department in the store.


latitude
Numerical
Geographic latitude of the store location.


longitude
Numerical
Geographic longitude of the store location.


market
Categorical
Broad geographic market where the order is being delivered.


order_city (order_delivery_city)
Categorical
City to which the order is being shipped.


order_country (order_delivery_country)
Categorical
Country where the order is delivered.


order_customer_id
Numerical
ID associated with the customer's order.


order_date
Datetime
Date when the order was placed.


order_id
Numerical
Unique code assigned to each order.


order_item_cardprod_id
Numerical
Product identifier scanned via RFID.


order_item_discount (item_discount_amount)
Numerical
Total discount amount applied to an item.


order_item_discount_rate (item_discount_rate)
Numerical
Discount as a percentage of the item’s price.


order_item_id
Numerical
Identifier for individual items in the order.


order_item_product_price (item_price_before_discount)
Numerical
Original price of the product before any discounts.


order_item_profit_ratio (item_profit_margin)
Numerical
Profit margin for each item in the order.


order_item_quantity
Numerical
Number of units purchased per item.


sales (total_sales)
Numerical
Total amount in sales generated.


order_item_total_amount (item_total_after_discount)
Numerical
Overall amount charged for the item, including quantity and discounts.


order_profit_per_order (post_discount_profit_per_order)
Numerical
Profit earned on a per-order basis after discount.


order_region
Categorical
Specific region where the order is shipped (e.g., Southeast Asia, Europe).


order_state (order_delivery_state)
Categorical
State or province of the delivery region.


order_status (order_current_status)
Categorical
Current status of the order (e.g., COMPLETE, CANCELED, ON_HOLD).


product_card_id
Numerical
Identifier for the product's card code.


product_category_id
Numerical
Category code linked to the product type.


product_name
Text
Full name or description of the product.


product_price (product_retail_price)
Numerical
Retail price of the product.


shipping_date
Datetime
Timestamp for when the item was shipped.


shipping_mode
Categorical
Type of shipping selected (e.g., Standard Class, Same Day).


label
Categorical
Delivery outcome: -1 for early, 0 for on time, 1 for delayed.


diesel_price
Numerical
Weekly diesel price.







In [ ]:





Import Libraries and Datasets¶




In [ ]:




In [6]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer, LabelEncoder, PowerTransformer, label_binarize
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay, auc
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import warnings
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.base import clone






In [ ]:




In [7]:


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Logistics Supply Chain Data Set
delivery = pd.read_csv('Logistics.csv')
print('\nFirst 5 Observations of the Delivery Dataset:')
display(delivery.head())

# Diesel Prices Supplemental Data
fuel = pd.read_excel('Diesel_Prices.xlsx', sheet_name='Data 1', skiprows=2)
fuel.columns = ['date', 'diesel_price']
print('\nFirst 5 Observations of the Weekly Diesel Prices Dataset:')
display(fuel.head())






In [ ]:






Methods¶Project Goal¶The primary goal of this project is to build a machine learning model that can predict delivery outcomes—specifically, whether an order will arrive early (-1), on time (0), or delayed (1). The classification will be based on features available prior to shipment, including product information, order metadata, shipping method, and geographic and customer-related attributes. This classification task is important because predicting delivery outcomes before an item is shipped allows businesses to proactively address potential issues in logistics. By identifying patterns that lead to delays or early deliveries, companies can optimize shipping decisions, adjust resource allocation, and improve customer satisfaction. In a supply chain environment where timeliness is crucial, even small improvements in delivery reliability can lead to significant operational and financial gains.
We plan to build and compare a Softmax Regression model, multiclass Random Forest and XGBoost models, and a binary version of whichever performs best among those three.
Data Description¶The main dataset used for this project consists of 15,549 rows and 41 columns, and it was obtained from Kaggle. The data includes information on customer orders placed between 2015 and 2018. Each row represents an item in an order, and the dataset contains both numerical and categorical features describing the order, the product, the customer, and the shipping process. There are 17 object, 1 integer, and 23 float variables before merging.
The target variable for this classification problem is:

label: Indicates the delivery outcome with three possible values:
-1: Order arrived earlier than expected
0: Order arrived on time
1: Order was delayed



However, we will eventually label encode these to where the values are only non-negative by increasing each by 1. Making it:

0: Order arrived earlier than expected
1: Order arrived on time
2: Order was delayed

Key predictor variables include:

shipping_mode: The method of shipping selected (e.g., Standard Class, Same Day).
order_region and order_country: Geographic indicators of where the order is being delivered.
order_item_product_price, order_item_discount, order_item_quantity: Financial and product-specific variables.
order_date and shipping_date: Timestamps that may help measure lead time or seasonal effects.
customer_segment: Type of customer (e.g., Consumer, Corporate, Home Office).
market: Broad geographic market classification (e.g., USCA, LATAM, Europe).
sales_per_customer and profit_per_order: Performance metrics that may correlate with order priority or shipping decisions.

Additionally, there appears to be an error in the original data dictionary provided on Kaggle, as the order_state will often be a state in the US, but the order_country and order_city won't be from the United States. This is likely because in this supply chain structure, it's common for orders to be registered at a store or fulfillment center located in one region (e.g., the United States) while the actual delivery destination is in another country (e.g., a LATAM region like Brazil or Mexico).
Additionally, I plan to integrate a secondary dataset that includes weekly U.S. diesel prices from 1993 to 2025, obtained from the U.S. Energy Information Administration. For orders delivered within the U.S., I will join diesel price data based on the week of the shipping date to explore whether fuel costs impact delivery reliability.
All variables have been reviewed and cleaned to ensure appropriate formatting, consistency in data types, and readiness for modeling. Categorical variables will be encoded, and numerical variables will be standardized or normalized as necessary during preprocessing.




In [ ]:





Merging Datasets¶To start, we will merge (left join) the supply chain data with the diesel prices, matching shipping date to the date of the diesel price by creating year-week formatted date for each dataset, then dropping it after merging.




In [ ]:




In [10]:


# Convert dates to datetime type
delivery['shipping_date'] = pd.to_datetime(delivery['shipping_date'], utc=True)
delivery['order_date'] = pd.to_datetime(delivery['order_date'], utc=True)
fuel['date'] = pd.to_datetime(fuel['date'])

# Create year-week columns for joining purposes
delivery['year_week'] = delivery['shipping_date'].dt.strftime('%Y-%U')
fuel['date'] = fuel['date'].dt.strftime('%Y-%U')

# Left join with the diesel prices df
merged_df = delivery.merge(fuel, how='left', left_on='year_week', right_on='date')
merged_df = merged_df.drop(columns=['year_week', 'date'])
merged_df = merged_df.sort_values('order_date') 






In [ ]:





Exploratory Data Analysis¶To get a better understanding of the data, we will plot some visualizations as well as some summary statistics below.




In [ ]:




In [12]:


merged_df['label_display'] = merged_df['label'].map({-1: 'Early', 0: 'On Time', 1: 'Delayed'})
id_cols = ['category_id', 'customer_id', 'customer_zipcode',
           'department_id', 'order_customer_id', 'order_id', 
           'order_item_cardprod_id', 'order_item_id', 'product_card_id',
           'product_category_id', 'label', 'label_display']
print('Table 2: Summary Statistics of Numerical Columns')
display(merged_df.drop(columns=id_cols, axis=1).describe())






In [ ]:




In [13]:


# Plot Figures 1-4 in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(17, 9))
fig.suptitle("Figures 1-4: Summary Visualizations", fontsize=16, y=0.95)

# Figure 1: Delivery Outcome
sns.countplot(data=merged_df, x='label_display', hue='label_display', ax=axes[0, 0])
axes[0, 0].set_title("Figure 1: Bar Plot of Delivery Outcome")
axes[0, 0].set_xlabel("Delivery Outcome")
axes[0, 0].set_ylabel("Frequency")
if axes[0, 0].get_legend():
    axes[0, 0].get_legend().remove()

# Figure 2: Shipping Mode By Delivery Outcome
sns.countplot(data=merged_df, x='shipping_mode', hue='label_display', ax=axes[1, 0])
axes[1, 0].set_title("Figure 2: Bar Plot of Shipping Mode By Delivery Outcome")
axes[1, 0].set_xlabel("Shipping Mode")
axes[1, 0].set_ylabel("Count")
axes[1, 0].tick_params(axis='x')
axes[1, 0].legend(title='Delivery Status')

# Figure 3: Skewness of Numerical Features
numerics = merged_df.drop(columns=id_cols).select_dtypes(include='number')
skew_df = numerics.apply(lambda x: x.skew()).sort_values(ascending=False)
sns.barplot(x=skew_df.values, y=skew_df.index, ax=axes[0, 1])
axes[0, 1].set_title("Figure 3: Bar Plot of Skewness of Numerical Features")
axes[0, 1].set_xlabel("Skewness")
axes[0, 1].set_ylabel("Feature")

# Figure 4: Destination Market By Delivery Status
sns.countplot(data=merged_df, x='market', hue='label_display', ax=axes[1, 1])
axes[1, 1].set_title("Figure 4: Bar Plot of Destination Market By Delivery Status")
axes[1, 1].set_xlabel("Market")
axes[1, 1].set_ylabel("Count")
axes[1, 1].legend(title='Delivery Status')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Figure 5: Store Location By Delivery Status
fig, ax = plt.subplots(figsize=(16, 5))
fig.suptitle("Figure 5: Bar Plot of Store Location By Delivery Status", fontsize=16)
sns.countplot(data=merged_df, x='customer_state', hue='label_display', ax=ax)
ax.set_xlabel("Store Location")
ax.set_ylabel("Count")
ax.legend(title='Delivery Status')
plt.tight_layout()
plt.show()

# Figure 6: Correlation Heatmap Plot
fig, ax = plt.subplots(figsize=(17, 6))
fig.suptitle("Figure 6: Correlation Heatmap Plot", fontsize=16)
sns.heatmap(merged_df.drop(columns=id_cols).select_dtypes(include='number').corr(),
            annot=True, cmap='coolwarm', ax=ax)
plt.tight_layout()
plt.show()






In [ ]:




In [14]:


# Count the occurrences of each class in 'label_display'
label_counts = merged_df['label_display'].value_counts()

# Identify majority and minority class sizes
majority_class = label_counts.max()
minority_class = label_counts.min()
imbalance_ratio = round(minority_class / majority_class, 3)

# Create DataFrame with imbalance ratio info
class_distribution = pd.DataFrame({
    'Count': label_counts
})
class_distribution['Imbalance vs Majority'] = round(label_counts / majority_class, 3)

print(class_distribution)
print(f"\nOverall Imbalance Ratio (Minority / Majority): {imbalance_ratio}")






In [ ]:





Figure 1 reveals a moderate class imbalance in the target variable, with delayed deliveries as the majority class. The smallest class, on-time deliveries, represents 33.7% of the delayed class. While this imbalance is notable, it remains within an acceptable range and does not justify the use of SMOTE or other resampling techniques, which could compromise model generalizability and real-world applicability. The overrepresentation of delayed deliveries may reflect underlying operational inefficiencies which we should keep.
The bar plot shown in Figure 2, illustrates the count of obsevations for each shipping mode by the delivery status. From it, we can deduce that whether a delivery will be early or not depends on the time alotted before it will be considered on-time or delayed, as the standard class which is the slowest type of shipping has the most amount of early arrivals by a significant amount. This reflects the more lenient time window given for orders with standard class shipping. Therefore, the shipping mode may prove to be vital in helping predict whether an order will be delayed, on time, or early.
Figure 3 shows that there exists some non-insignifcant skewness in some of the numeric features, most prominently the profit per order, profit per order after discount (order_profit_per_order), and the order item discount. We should likely apply a log transformation on all of these features asides from the item quantity, discount rate, diesel price, latitude, and longitude. While not having skewness isn't an assumption for logistic regression or more complex ML models, it can hinder performance in logistic regression models at the very least and should be addressed.
In Figure 4 and Figure 5, we can see the frequency of where orders are delivered from and to, with Figure 4 showing a bar plot of where most of the deliveries come from and help indicate where this supply chain company is located in, relatively speaking. We can see they primarily work in Puerto Rico, California, and New York along with a few other states in the US. However, Figure 5 shows that most of their deliveries are to Europe and Latin America. This suggests the company is structured where orders are registered through U.S. stores or fulfillment centers but delivered internationally, indicating a centralized processing hub serving cross-border customers. This helps explain why there are so many delays in the dataset due to the distance of the deliveries being international rather than local.
Another note to make is there appears to be an invalid value in Figure 4, as there is a store location (customer_state) listed as '91732', it is very likely a zip code and we can determine the state of that zip code and replace it with the actual state abbreviation. In this case, the zip code is in California (CA). We will perform this during the data preprocessing.
The correlation heatmap in Figure 6 helps showcase which numerical features are correlated with each other and we can extract a few useful insights from it. The most important one is there being multicollinearity among some of the features, particularly ones regarding the price and discount as there is an original price as well as the price after applying the discount, which should lead to it being correlated. Because logistic regression assumes no multicollinearity and we are focused on prediction, we will apply an L2 penalty in logistic regression, shrinking irrelevant or correlated predictors close to zero but not eliminating it entirely as some of these features may still be informative.
We will now begin preprocessing the data set for modeling.




In [ ]:





Preprocessing¶First we will begin by extracting information from the date of the order and shipping, however, we need to check if there are any invalid rows where the shipping occurs before the order is placed.




In [ ]:




In [17]:


print("Invalid rows:", (merged_df['shipping_date'] < merged_df['order_date']).sum())






In [ ]:





There is in fact about 5941 rows that are invalid, making this appear systematic. Since this is a good chunk of data to lose if we were to drop them, we will instead create a new column to flag a row as 1 if invalid, otherwise 0 and set the days between the two dates column to be NaN if that's the case and allow the median imputation to handle it. Now, we will extract the rest of the info we can from the dates.
We will also feature engineer several features using the order date and shipping date. These include:

Day of the week when order was placed
Month when order was placed
Year order was placed
Whether the order was placed on a weekend or not
Days between the order and shipping date
An indicator of whether the date flow is invalid (where shipping date is before order date by mistake)

Finally we relabeled our columns so when we extract the features for feature importance it's easier to distinguish which columns have to do with the discount and which have to do without the discount.




In [ ]:




In [19]:


merged_df['order_dayofweek'] = merged_df['order_date'].dt.dayofweek
merged_df['order_month'] = merged_df['order_date'].dt.month
merged_df['order_year'] = merged_df['order_date'].dt.year
merged_df['is_weekend'] = merged_df['order_dayofweek'].isin([5, 6]).astype(int)

merged_df['days_between_order_and_ship'] = (merged_df['shipping_date'] - merged_df['order_date']).dt.days
merged_df['invalid_date_flow'] = (merged_df['shipping_date'] < merged_df['order_date']).astype(int)
merged_df.loc[merged_df['days_between_order_and_ship'] < 0, 'days_between_order_and_ship'] = np.nan

# Relabeling
column_relabel = {
    'profit_per_order': 'pre_discount_profit_per_order',
    'order_profit_per_order': 'post_discount_profit_per_order',
    'sales_per_customer': 'total_sales_per_customer',
    'order_item_product_price': 'item_price_before_discount',
    'order_item_total_amount': 'item_total_after_discount',
    'product_price': 'product_retail_price',
    'sales': 'total_sales',
    'order_item_discount': 'item_discount_amount',
    'order_item_discount_rate': 'item_discount_rate',
    'order_item_profit_ratio': 'item_profit_margin',
    'order_status': 'order_current_status',
    'order_state': 'order_delivery_state',
    'order_country': 'order_delivery_country',
    'order_city': 'order_delivery_city',
    'customer_country': 'customer_purchase_country',
    'customer_city': 'customer_purchase_city',
    'customer_state': 'store_location_state',
}

merged_df = merged_df.rename(columns=column_relabel)
print('Column Names After Relabeling:\n' + str(merged_df.columns))






In [ ]:





Now that we have successfully relabeled the column names for clarity, let's check to see which columns specifically have missing values and then plot the distribution of those columns to determine whether we should use mean or median imputation based on the skewness.




In [ ]:




In [21]:


na_counts = merged_df.isnull().sum()
print(f'Sum of NAs in Columns with at least 1 NA:\n{na_counts[na_counts > 0]}') # Show sum of NAs for columns with at least 1 NA

print()
# Histogram 1: Days Between Order and Ship
fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].hist(merged_df['days_between_order_and_ship'], bins=20, edgecolor='black')
axes[0].set_title("Days Between Order and Shipping Date")
axes[0].set_xlabel("Days Between Order and Ship")
axes[0].set_ylabel("Frequency")

# Histogram 2: Diesel Price
axes[1].hist(merged_df['diesel_price'], bins=10, edgecolor='black')
axes[1].set_title("Diesel Price Distribution")
axes[1].set_xlabel("Diesel Price")
axes[1].set_ylabel("Frequency")
fig.suptitle("Figure 7 & 8: Histograms of Order Processing and Diesel Prices", fontsize=14)
fig.tight_layout()
plt.show()






In [ ]:





We have 66 missing values in diesel price, we will impute those values with the median price. We will also replace the invalid value in customer_state of '91732' with 'CA', since the zipcode is in California according to Google. Next we will log transform skewed predictors that have no negative values with a small added constant, onehot encode categorical predictors, ordinal encode shipping_mode, and then finally standardize all remaining numeric predictors. To do the log transformation with an added constant we will create a simple function called safe_log1p(). In addition, for the days between order and shipping date, we will impute the NAs using the median because the values are skewed right.
For columns that are skewed but contain negative values, we will do a Yeo-Johnson transformation instead, which is similar to a Box-Cox transformation. Unlike Box-Cox, which is limited to strictly positive data, the Yeo-Johnson transformation can handle zero and negative values, making it a more flexible option for real-world datasets. It applies a power transformation that stabilizes variance and makes the data more normally distributed, improving the performance of many machine learning models. By adjusting the shape of the distribution based on the data's characteristics, Yeo-Johnson ensures that features with a wide or asymmetric spread—including those spanning below zero—can be normalized effectively without the need for arbitrary shifts or imputations.
For the target variable, label, we will label encode it turning it from (-1, 0, 1) to (0, 1, 2).
To prepare the dataset for modeling, several data cleaning and transformation steps were performed:

66 NaNs in diesel_price were imputed using the mean, due to the distribution being normal/symmetrics.

5941 NaNs in days_between_order_and_ship were imputed using the median, as the distribution was found to be right-skewed.

An invalid entry '91732' found in the customer_state column was corrected to 'CA', as this ZIP code corresponds to a location in California.

For numerical features with positive skew and no negative values, a log transformation with an added constant of 1e-6 was applied using a custom function safe_log1p() to reduce skewness and stabilize variance.

Applied Yeo-Johnson transformation on skewed numerical features with negatives. Unlike the Box-Cox or log transformation, Yeo-Johnson handles zero and negative values, making it more adaptable for real-world data. It adjusts the shape of the distribution to normalize features effectively without the need for data shifting, which helps improve the performance of various machine learning models.

Categorical variables with limited cardinality were one-hot encoded to allow inclusion in machine learning models while preventing high-dimensionality issues.

We did not include nominal features with many levels to avoid significantly increasing dimensionality.

The shipping_mode feature was ordinally encoded based on a logical ranking: ['Standard Class', 'Second Class', 'First Class', 'Same Day'].

All numeric predictors were standardized using StandardScaler() to ensure consistent scaling across features.

The target variable label, originally consisting of the values -1, 0, and 1, was label-encoded into 0, 1, and 2 respectively, to make it compatible with scikit-learn's classification algorithms.






In [ ]:




In [23]:


merged_df_copy = merged_df.copy() # Make a copy for merged_df to let XGBoost handle NAs
merged_df['store_location_state'] = merged_df['store_location_state'].replace('91732', 'CA')
merged_df['diesel_price'] = merged_df['diesel_price'].fillna(merged_df['diesel_price'].mean())
merged_df['days_between_order_and_ship'] = merged_df['days_between_order_and_ship'].fillna(merged_df['days_between_order_and_ship'].median())

# Custom log to ensure negatives don't cause errors
def safe_log1p(X):
    return np.log1p(np.nan_to_num(X)) + 1e-6

transform_cols = [
    'item_discount_amount', 'item_price_before_discount', 'product_retail_price',
    'total_sales_per_customer', 'total_sales', 'item_total_after_discount',
    'item_profit_margin', 'pre_discount_profit_per_order', 'post_discount_profit_per_order',
    'days_between_order_and_ship'
]

safe_log_cols = [col for col in transform_cols if (merged_df[col] >= 0).all()]
yeo_johnson_cols = [col for col in transform_cols if (merged_df[col] < 0).any()]

print('Columns to Log:', safe_log_cols)
print("Columns for Yeo-Johnson Transform:", yeo_johnson_cols)

ct = ColumnTransformer(
    [("log", FunctionTransformer(safe_log1p, feature_names_out='one-to-one'),
      safe_log_cols),
     ("yeojohnson", PowerTransformer(method='yeo-johnson'), yeo_johnson_cols),
     ("ordinal", OrdinalEncoder(categories=[['Standard Class', 'Second Class', 'First Class', 'Same Day']],
                                handle_unknown='use_encoded_value', unknown_value=-1), 
      ['shipping_mode']),     
     ("onehot", OneHotEncoder(drop='first'),
      ['payment_type', 'customer_purchase_country', 'customer_segment', 'department_name',
       'market', 'order_region', 'order_current_status', 'order_dayofweek', 'order_month'])], 
    remainder='passthrough', verbose_feature_names_out=False
)






In [ ]:





We have selected columns that not only do we believe to be important but also categorical features that do not have many levels to avoid adding dimensionality to the data, as several categorical columns contained hundreds to thousands of levels. Furthermore, these high-level factor variables were redundant in that other features contained similar information while adding less dimensionality from onehot encoding. For example, customer_purchase_city, has 555 levels, whereas customer_purchase_country has only 2 levels and market has 5 levels and still cover similar information.
After initial feature selection, we have retained 28 predictors before onehot and ordinal encoding.
Now that all the preprocessing is done, we will now perform a 70/30 training-test split stratified on the delivery outcome (label).




In [ ]:





Train/Test Split¶




In [ ]:




In [26]:


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(merged_df['label'])

# Select only relevant and not redundant non-ID columns
X = merged_df.iloc[:, [0, 1, 2, 6, 8, 12, 13, 14, 15, 22, 
                       23, 25, 26, 27, 28, 29, 30, 31, 33, 
                       37, 39, 41, 43, 44, 45, 46, 47, 48]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, train_size=.7, random_state=1, stratify=y)






In [ ]:




In [27]:


# Fit transformer on training data
ct.fit(X_train)

# Get total number of output features
num_transformed_features = len(ct.get_feature_names_out())
print(f"Number of predictors after ColumnTransformer: {num_transformed_features}")






In [ ]:





After preprocessing, we will have 84 predictors, while this is much less than if we had kept all columns, we should still be careful about overfitting and regularize when applicable in the models.
We can now begin the modeling. The models we will build are:

Softmax Regression
Multiclass Random Forest
Multiclass XGBoost
Binary version of the best performing multiclass model






In [ ]:





Results¶Model Assumptions¶Now we will check the necessary assumptions for each of the models listed below.

Logistic Regression
Linearity of Log-Odds
Multicollinearity: We will address this by tuning the L2 penalty parameter
Outliers: Log-transformation and Yeo-Johnson to help mitigate outliers
Independence of observations: True by default, as one order’s delivery outcome doesn't affect another's.


Random Forest
Independence of observations: True
Sufficient data: True, since we have more than 10 times the number of features in rows


XGBoost
Independence of observations: True
Sufficient data: True







In [ ]:




In [30]:


# Linearity of Log-Odds assumption check
X_ct = ct.fit_transform(X_train)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_ct)
model = LogisticRegression(max_iter=1000)
model.fit(X_scaled, y_train)
probs = model.predict_proba(X_scaled)[:, 1]
logit = np.log(probs / (1 - probs))

feature_names = ct.get_feature_names_out()
selected_features = ['item_profit_margin', 'total_sales_per_customer', 'product_retail_price']
df_plot = pd.DataFrame({
    feat: X_scaled[:, [j for j, name in enumerate(feature_names) if feat in name][0]]
    for feat in selected_features
})
df_plot['logit'] = logit
df_plot['label'] = y_train

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle("Figure 9: Linearity Check of Logit vs Selected Predictors (Scaled)", fontsize=14)
for ax, feat in zip(axes, selected_features):
    sns.scatterplot(data=df_plot, x=feat, y='logit', hue='label', ax=ax, alpha=0.6)
    ax.set_title(f"{feat.replace('_', ' ').title()}")
    ax.set_xlabel(f"{feat.replace('_', ' ').title()}")
    ax.set_ylabel("Logit")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()






In [ ]:





For brevity, we plotted the logit against only a few continuous predictors as seen in Figure 9, in which there is insufficient visual evidence to meet the linearity of log-odds assumption for logistic regression. Instead, our predictors seem to follow a uniform distribution, suggesting a nonlinear model may be better suited. Therefore our logistic regression model will not be interpretable, but can still be used for prediction which is still suitable for goal, as we are prioritizing prediction (performance) over interpretation. We can expect the softmax regression model to underperform due to many features not being linearly related to the logit of the delivery outcome. However, we can still interpret the random forest and XGBoost models due to having their assumptions met.




In [ ]:





Multinomial Classification¶We can now finally begin building our models, starting with logistic regression, followed by random forest, and lastly a boosted model. All three models will incorporate hyperparameter tuning using 5-fold cross-validation, stratified by the delivery outcome.
We begin by building multiclass models to predict each delivery outcome. Subsequently, we will binarize the target variable into 'delayed' (1) and 'not delayed' (0), and retrain the best-performing model from the multiclass setting for binary classification before ultimately comparing across all models.
Our pipeline will consist of the column transformations specified earlier, followed by a mean imputation step applied to any remaining numeric variables. Although our current dataset has already addressed missing values using manual mean and median imputation, this step is included in the pipeline to ensure robustness. If the model is reused in the future on new data containing missing values, this imputation step will automatically handle them and maintain model stability. Finally, all predictors will be standardized to prepare the data for modeling.
Although we care most about capturing as many delays as possible, we used recall_macro as the scoring metric for stratified k-fold cross-validation. By treating all delivery outcomes equally, we prevent the model from achieving high recall merely by predicting delayed deliveries, which represent the majority class. This approach ensures balanced sensitivity across early, on-time, and delayed classes.




In [ ]:




In [33]:


def evaluate_model(display_name, y_test, y_pred):
    print(f"\n{display_name}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=3))

def run_grid_search_pipeline(display_name, pipeline, param_grid, X_train, y_train, X_test, y_test):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    grid = GridSearchCV(pipeline, param_grid=param_grid, cv=cv, n_jobs=-1, verbose=1, error_score='raise', scoring='recall_macro')
    grid.fit(X_train, y_train)
    print(f"\n{display_name} Best Params: {grid.best_params_}")
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    evaluate_model(display_name, y_test, y_pred)
    return best_model






In [ ]:





Softmax Regression¶




In [ ]:




In [35]:


lr_pipeline = Pipeline([("preprocess", ct),
                        ("impute", SimpleImputer(strategy="mean")),
                        ("scale", StandardScaler()),
                        ("lr", LogisticRegression(random_state=1, solver='lbfgs', max_iter=1000, n_jobs=-1))])
lr_param_grid = {'lr__C': np.logspace(-4, 4, 50)}
best_lr = run_grid_search_pipeline("Softmax Regression", lr_pipeline, lr_param_grid, X_train, y_train, X_test, y_test)






In [ ]:





The best hyperparameter for the softmax regression model was C = 24.42. In multinomial logistic regression, C controls the inverse of regularization strength. A relatively high C indicates that the model performed best with weaker regularization, allowing more flexibility in fitting the class boundaries between the multiple delivery outcomes (early, on-time, delayed). This suggests that the model benefits from retaining more of the signal in the data, possibly due to informative and well-preprocessed features.
Softmax Regression performed well in identifying delayed deliveries (Recall: 88.9%) and moderately on early deliveries (Recall: 31.2%), but failed to detect on-time deliveries (Recall: 0.1%), resulting in modest overall accuracy (58.5%) and low macro-average recall (40.1%).




In [ ]:





Random Forest¶




In [ ]:




In [38]:


rf_pipeline = Pipeline([("preprocess", ct),
                        ("impute", SimpleImputer(strategy="mean")),
                        ("scale", StandardScaler()),
                        ("rf", RandomForestClassifier(random_state=1, n_jobs=-1))])
rf_param_grid = {'rf__n_estimators': [25, 50, 75, 100], 'rf__max_depth': [15, 20, 25], 'rf__min_samples_split': [2, 3, 4]}
best_rf = run_grid_search_pipeline("Random Forest", rf_pipeline, rf_param_grid, X_train, y_train, X_test, y_test)






In [ ]:





The best-performing Random Forest used 50 trees, a maximum depth of 25, and required at least 4 samples to split a node. This configuration suggests the model benefited from moderately deep trees with slightly more conservative splitting, likely helping to reduce overfitting while still capturing complex patterns in the data.
Random Forest achieved strong recall on delayed deliveries (85.1%) and moderate performance on early deliveries (Recall: 41.9%), but performed poorly on on-time deliveries (Recall: 1.9%), resulting in overall accuracy of 59.0% and unbalanced class performance.




In [ ]:





XGBoost¶




In [ ]:




In [41]:


xgb_pipeline = Pipeline([("preprocess", ct), 
                         ("scale", StandardScaler()), 
                         ("xgb", XGBClassifier(objective='multi:softprob', random_state=1, n_jobs=-1, eval_metric='mlogloss'))])
xgb_param_grid = {'xgb__n_estimators': [30, 55, 90, ], 'xgb__max_depth': [4, 5, 6], 'xgb__learning_rate': [0.005, 0.01, 0.05], 'xgb__subsample': [0.7, 0.8, 0.9]}
best_xgb = run_grid_search_pipeline("XGBoost", xgb_pipeline, xgb_param_grid, X_train, y_train, X_test, y_test)






In [ ]:





The best-performing XGBoost model used a low learning rate (0.01), shallow trees (max depth = 4), and fewer boosting rounds (30 estimators) with a high subsample rate (90%). This configuration indicates that the model prioritized conservative, gradual learning with broad data coverage to improve generalization and reduce overfitting.
XGBoost showed the most balanced multiclass performance, with strong recall on delayed deliveries (78.9%) and early deliveries (62.0%), while moderately improving on-time recall (6.6%). Given this relative balance, it will serve as the basis for the binary classification comparison.




In [ ]:





Binary Classification¶




In [ ]:





To improve generalizability and mitigate the slight class imbalance observed in the multiclass setting, we now explore a binary classification approach. Instead of predicting three separate delivery outcomes (early, on-time, delayed), we consolidate early and on-time deliveries into a single class labeled as 0 (not delayed), while delayed deliveries are labeled as 1 (delayed). This binarization simplifies the prediction task and focuses model learning on distinguishing between acceptable and problematic outcomes from a business perspective.
This approach is particularly valuable because the on-time class was severely underrepresented in predictions, leading to poor recall in the multiclass models. By grouping early and on-time outcomes—both of which are operationally satisfactory—we reduce noise and ambiguity in the target variable, making it easier for the model to learn a clear distinction between satisfactory and unsatisfactory deliveries.
Moreover, from a business standpoint, the primary concern is identifying and reducing delayed shipments, as they are most likely to affect customer satisfaction and supply chain efficiency. For this reason, we adjust the scoring metric used during hyperparameter tuning to focus on recall, prioritizing the model's ability to correctly identify as many delayed deliveries as possible, even if it comes at the expense of precision.




In [ ]:




In [45]:


def evaluate_model_binary(display_name, y_test, y_pred):
    print(f"\n{display_name}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=3))
def run_grid_search_pipeline_binary(display_name, pipeline, param_grid, X_train, y_train, X_test, y_test, scoring='roc_auc'):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    grid = GridSearchCV(pipeline, param_grid=param_grid, cv=cv, scoring=scoring, n_jobs=-1, verbose=1, error_score='raise')
    grid.fit(X_train, y_train)
    print(f"\n{display_name} Best Params: {grid.best_params_}")
    y_pred = grid.best_estimator_.predict(X_test)
    evaluate_model_binary(display_name, y_test, y_pred)
    return grid.best_estimator_






In [ ]:





XGBoost (Binary)¶




In [ ]:




In [47]:


y = (merged_df_copy['label'] == 1).astype(int).values
X = merged_df_copy.iloc[:, [0, 1, 2, 6, 8, 12, 13, 14, 15, 22, 
                       23, 25, 26, 27, 28, 29, 30, 31, 33, 
                       37, 39, 41, 43, 44, 45, 46, 47, 48]]
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, y, test_size=.3, train_size=.7, random_state=1, stratify=y)
xgb_pipeline = Pipeline([("preprocess", ct), ("scale", StandardScaler()), ("xgb", XGBClassifier(objective='binary:logistic', random_state=1, n_jobs=-1, eval_metric='logloss'))])
xgb_param_grid = {'xgb__n_estimators': [30, 55, 90], 'xgb__max_depth': [4, 6], 'xgb__learning_rate': [0.01, 0.05], 'xgb__subsample': [0.8, 0.9], 'xgb__colsample_bytree': [0.8, 0.9, 1]}
best_xgb_binary = run_grid_search_pipeline_binary("XGBoost (Binary Classification)", xgb_pipeline, xgb_param_grid, X_train_binary, y_train_binary, X_test_binary, y_test_binary)






In [ ]:





The XGBoost binary classifier achieved an accuracy of 69.0%, with a recall of 81.1% and precision of 70.0% for the delayed class. The macro-averaged recall was 66.8% and macro-averaged precision was 68.6%. The best hyperparameters identified were: colsample_bytree = 1, learning_rate = 0.01, max_depth = 4, n_estimators = 55, and subsample = 0.9.




In [ ]:






Discussion¶We will compare the models using their metrics and confusion matrices, followed by the ROC curve and AUC plot of the best-performing model, and finally the top 30 feature importances—each accompanied by a focused discussion of its findings.
Model Comparison¶




In [ ]:




In [50]:


# Confusion Matrices Display
models = [("Softmax Regression", best_lr), ("Random Forest", best_rf), ("XGBoost (Multiclass)", best_xgb), ("XGBoost (Binary)", best_xgb_binary)]
fig, axes = plt.subplots(1, len(models), figsize=(4 * len(models), 4))
for ax, (name, model) in zip(axes, models):
    if "Binary" in name:
        y_true = y_test_binary; y_pred = model.predict(X_test_binary)
    else:
        y_true = y_test; y_pred = model.predict(X_test)
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred, cmap="Blues", ax=ax); ax.set_title(name); ax.grid(False)
plt.tight_layout(); plt.subplots_adjust(top=0.8); plt.suptitle('Figure 10: Model Confusion Matrices', fontsize=14); plt.show()






In [ ]:





Table 2: Model Metrics Comparison¶


Model
Accuracy
Recall
Precision
Recall (Early)
Recall (On‑Time)
Recall (Delayed)
Recall (Not Delayed)
Precision (Early)
Precision (On‑Time)
Precision (Delayed)
Precision (Not Delayed)




Softmax Regression
0.585
0.401
0.513
0.312
0.001
0.889
—
0.419
0.500
0.619
—


Random Forest
0.590
0.430
0.439
0.419
0.019
0.851
—
0.434
0.239
0.642
—


XGBoost
0.610
0.492
0.576
0.620
0.066
0.789
—
0.440
0.594
0.694
—


XGBoost (Binary) 🏆
0.690
0.668
0.686
—
—
0.811
0.525
—
—
0.700
0.671



Looking at Table 2 and Figure 10, the XGBoost Binary classifier was the strongest performer for the project's goal: early and accurate detection of delayed deliveries. It achieved the highest recall for delayed shipments (81.1%) and a strong precision (70.0%), making it the most effective model for minimizing undetected delays. Its confusion matrix shows that a large number of true delayed deliveries (class 1) were correctly identified, while false positives for delays remained moderate. This demonstrates the model’s ability to identify most delayed orders—while maintaining respectable precision—enabling proactive mitigation strategies, even at the cost of some false alarms. Additionally, its overall accuracy (69.0%) and balanced performance confirm its practical utility in a real-world logistics environment.
In the multiclass setting, the standard XGBoost model demonstrated the best overall multiclass performance, with the highest accuracy (61.0%) and the most balanced combination of recall (49.2%) and precision (57.6%) across the three delivery classes. As seen in its confusion matrix, the model correctly identified a majority of delayed shipments and performed well in detecting early deliveries (recall: 62.0%), but again struggled with identifying on-time deliveries, achieving only 6.6% recall. A large number of on-time deliveries were misclassified as delayed or early. Despite this limitation, the model’s strength in distinguishing extreme outcomes—early versus delayed—could still provide strategic value in scenarios where proactive planning hinges more on detecting outliers than exact timing.
The Random Forest model showed moderate performance, with delay recall at 85.1%, slightly outperforming XGBoost in that aspect. However, its early (41.9%) and on-time (1.9%) recall were much lower. The confusion matrix for this model illustrates the severe misclassification of on-time deliveries, which are almost entirely absorbed into the delayed class. Precision across all classes is inconsistent, suggesting that while the model effectively identifies late deliveries, it lacks nuance in differentiating early and on-time cases. This limits its usefulness in a multiclass classification context where more granular insights are needed.
The Softmax Regression model performed weakest overall, with the lowest accuracy (58.5%) and the lowest macro recall (40.1%). It particularly struggled with on-time recall, achieving a mere 0.1%, essentially failing to detect this class. As shown in its confusion matrix, most on-time deliveries were predicted as delayed, leading to significant class imbalance in the predictions. While its delay recall (88.9%) is high and precision on early deliveries (41.9%) is relatively strong, its inability to handle all three classes effectively underscores the limitations of using a simpler linear model for this type of problem.
In summary, the confusion matrices (Figure 10) visually support and reinforce the performance metrics shown in Table 2, highlighting the per-class strengths and trade-offs of each model. While the multiclass XGBoost model demonstrated better balance overall, its poor on-time detection detracts from its utility in real-world settings. The binary XGBoost model, by simplifying the target to delayed versus not delayed, successfully maximized recall on the most important class and maintained strong precision—making it best suited for business-critical delay prediction.




In [ ]:





ROC Curve and AUC¶Now that we have chosen the final model (XGBoost binary model), we will look deeper into the ROC curve and the AUC to get a better understanding of it's performance before proceeding to the feature analysis.




In [ ]:




In [53]:


# ROC Curve Plot w/ AUC
y_proba = best_xgb_binary.predict_proba(X_test_binary)[:, 1]
fpr, tpr, _ = roc_curve(y_test_binary, y_proba)
auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6)); plt.plot(fpr, tpr, label=f'AUC = {auc:.3f}')
plt.plot([0, 1], [0, 1], 'k--', label='Random Baseline'); plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('Figure 11: ROC Curve of XGBoost Binary Model', fontsize=14)
plt.legend(loc='lower right'); plt.grid(True); plt.show()






In [ ]:





Figure 11 shows the ROC curve for the binary XGBoost model, used to predict whether a delivery will be delayed. The model achieves an AUC (Area Under the Curve) of 0.777, indicating good overall classification performance. An AUC closer to 1.0 reflects a stronger ability to distinguish between delayed and non-delayed shipments, while 0.5 would represent random chance.
The ROC curve rises steeply at the beginning, showing that the model captures a high true positive rate (recall) at relatively low false positive rates. This is valuable in the logistics context, where correctly identifying delayed shipments is more critical than occasionally flagging a non-delayed one. The model is therefore effective at detecting most delays early without a high cost of false alarms.
The diagonal line represents a random classifier. The XGBoost curve consistently stays above this baseline, confirming meaningful predictive skill. Overall, the model balances sensitivity and specificity well and is a strong candidate for real-world deployment where early detection of delays is essential for proactive operational response.




In [ ]:





Feature Analysis¶Now that we have chosen the final model (XGBoost binary model), we will perform a feature analysis on the most important predictors according to the model. This will give us a better understanding of which features are important and which can be safely removed with no loss in model performance. Only the top 30 features will be displayed for brevity.




In [ ]:




In [56]:


# Get Feature Names and Importances
feature_names = ct.get_feature_names_out()
xgb_model = best_xgb_binary.named_steps['xgb']
importances = xgb_model.feature_importances_

# Sort by top 30 importances
indices = np.argsort(importances)[::-1]
sorted_features = np.array(feature_names)[indices]
sorted_importances = importances[indices]
top_n = 30
top_features = sorted_features[:top_n]
top_importances = sorted_importances[:top_n]
top_features_reversed = top_features[:top_n][::-1]
top_importances_reversed = top_importances[:top_n][::-1]

# Plot
plt.figure(figsize=(10, 7))
sns.set(style="whitegrid")
bars = plt.barh(range(top_n), top_importances_reversed, align='center', color=sns.color_palette("Blues", top_n))
plt.yticks(range(top_n), top_features_reversed, fontsize=10)
plt.xlabel("Importance Score", fontsize=12)
plt.title("Figure 12: Top 30 Feature Importances from XGBoost (Binary Classification)", fontsize=14)
for i, bar in enumerate(bars):
    plt.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2,
             f"{top_importances_reversed[i]:.3f}", va='center', fontsize=9)
plt.tight_layout()
plt.show()






In [ ]:





The feature importance plot shown in Figure 12 reveals that shipping_mode overwhelmingly dominates the model, accounting for 66% of the total importance across all 84 predictors. This is consistent with domain expectations—shipping services with wider delivery windows (e.g., Standard Class) are naturally less likely to be flagged as delayed compared to express modes with stricter timing.
Furthermore, diesel_price, sourced from the U.S. Energy Information Administration, ranks as the 8th most important feature, indicating that macro-level transportation cost fluctuations contributed modestly to model performance—more so than most other variables.
Other moderately important features include days_between_order_and_ship, invalid_date_flow, and longitude, each contributing around 2–3%. These features likely reflect latent temporal or geographic delivery dynamics, such as inefficient processing gaps or regional shipping differences.
A long tail of low-importance features follows, including many one-hot encoded indicators like customer_segment_Home Office and payment_type_PAYMENT. While individually minor, their presence suggests the model captures some behavioral or operational patterns.
Interestingly, most transactional features—including profit margins, item discounts, and sales totals—contributed very little to the model. This emphasizes that operational metadata (e.g., logistics details, timing, and geography) is more predictive of delivery delays than financial or sales-related variables.
Additionally, multiple order_region_* features show small but roughly equal importance, suggesting mild geographic variation without any single region driving the prediction.
With 84 predictors in total, the steep drop-off in importance after the top 2–3 features highlights strong potential for feature reduction. Strategies such as removing weak predictors, consolidating sparse categorical levels, or using PCA on less informative variables could simplify the model without significant loss in performance.
Lastly, we can also replace many of these unimportant features with columns we decided to remove due to dimensionality concerns and belief they would be redundant.




In [ ]:





Next Steps¶While the models developed in this project demonstrated moderate to strong predictive performance—particularly the binary XGBoost classifier—there are several avenues for further refinement and enhancement based on the feature importance findings.
First, feature selection and engineering could be optimized further. The feature importance plot revealed that only a small subset of predictors—most notably shipping_mode—accounted for the majority of the model’s predictive power. This suggests strong potential for feature pruning, where low-importance features can be removed to streamline the model and reduce complexity without sacrificing accuracy. At the same time, new features may be engineered to capture additional structure, such as interaction terms or lag-based temporal variables (e.g., rolling averages of delivery performance).
Additionally, while diesel price contributed modestly as the 8th most important predictor, its impact opens the door to incorporating other fuel-based macroeconomic indicators. In particular, jet fuel prices could be explored in future iterations, as Figures 4 and 5 suggest that international deliveries are common. Including jet fuel trends could provide stronger signal for air-based shipments, especially in overseas contexts.
Another promising direction involves reducing the number of levels in categorical variables. Several categorical features—like customer segments, order regions, and store locations—appeared with marginal yet redundant influence. Consolidating rare or overlapping levels into broader categories could reduce dimensionality and improve generalization, especially for one-hot encoded variables that expand the feature space unnecessarily.
Probability calibration may also improve performance. Techniques such as Platt scaling or isotonic regression can make predicted probabilities more reliable and actionable. This is especially relevant in the binary classification setting, where recall is prioritized to capture as many delayed deliveries as possible.
Lastly, incorporating unsupervised learning techniques could add predictive value. For example, applying K-Means clustering to PCA-reduced features may uncover latent structures in the data, such as underlying order patterns or customer segments. The resulting cluster labels can then be added as categorical features, helping the model capture complex, nonlinear relationships that may not be apparent through raw variables alone.
Overall, the final model performs well, but additional refinements in feature engineering, dimensionality reduction, and probability calibration have the potential to further improve its predictive accuracy and practical impact.





In [ ]:





Conclusion¶This machine learning project aimed to predict delivery outcomes—early, on-time, or delayed—using real-world logistics data, enriched with external diesel price trends. Through feature engineering and model tuning, multiple classification algorithms were evaluated. Among them, the XGBoost Binary model emerged as the most reliable, offering the highest recall on delayed deliveries. This capability is especially critical in logistics, where failing to anticipate delays can lead to operational disruptions and customer dissatisfaction.
While models like Random Forest and multiclass XGBoost offered nuanced classifications, they struggled with on-time delivery detection and required more complexity for marginal gains. Simpler models like Softmax Regression lacked the capacity to generalize effectively across classes.
Ultimately, this work demonstrates that predictive modeling—when properly tuned and evaluated—can offer real, actionable value in managing supply chain reliability. The selected model enables smarter decision-making, anticipatory logistics planning, and improved customer experience. Furthermore, with the proper additions specified in above such as calibration, feature engineering clusters via K-Means, and further feature selection, the final model can likely be improved even more.




In [ ]:






References¶
Gabellini, M., Civolani, L., Calabrese, F., & Bortolini, M. (2024). A Deep Learning Approach to Predict Supply Chain Delivery Delay Risk Based on Macroeconomic Indicators: A Case Study in the Automotive Sector. Applied Sciences, 14, 4688. https://doi.org/10.3390/app14114688

Ivanov, D., & Dolgui, A. (2020). A Digital Supply Chain Twin for Managing the Disruption Risks and Resilience in the Era of Industry 4.0. Production Planning & Control, 32, 775–788. https://doi.org/10.1080/09537287.2020.1768450

Pazoki, M., Samarghandi, H., & Behroozi, M. (2023, August 1). Increasing Supply Chain Resiliency Through Equilibrium Pricing and Stipulating Transportation Quota Regulation. https://doi.org/10.48550/arXiv.2308.00681

Rokoss, A., Syberg, M., Tomidei, L., Hülsing, C., Deuse, J., & Schmidt, M. (2024). Case Study on Delivery Time Determination Using a Machine Learning Approach in Small Batch Production Companies. Journal of Intelligent Manufacturing, 35, 3937–3958. https://doi.org/10.1007/s10845-023-02290-2


Data Sources¶
Administration, U. E. (2025, March 17). Weekly Retail Gasoline and Diesel Prices. United States. Retrieved March 23, 2025, from https://www.eia.gov/dnav/pet/pet_pri_gnd_dcus_nus_w.htm

Kamboj, P. (2024, September 2). Logistics Supply Chain Real World Data. Retrieved from https://www.kaggle.com/datasets/pushpitkamboj/logistics-data-containing-real-world-data




